In [7]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
import openai
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.chat_models import ChatOpenAI
import yfinance as yf


In [20]:
def get_yfinance(ticker): 
    stock = yf.Ticker(ticker)
    stock.info
    print(stock.info)

    # print stock history and metadata
    hist = stock.history(period = "1mo")
    print(stock.history_metadata)
    print(hist)

    # show actions (dividends, splits, capital gains)
    print(stock.actions)
    print(stock.dividends)
    print(stock.splits)
    #stock.capital_gains  only for mutual funds & etfs


    # show financials:
    print(stock.calendar)
    print(stock.sec_filings)
    # - income statement
    print(stock.income_stmt)
    print(stock.quarterly_income_stmt)
    # - balance sheet
    print(stock.balance_sheet)
    print(stock.quarterly_balance_sheet)
    # - cash flow statement
    print(stock.cashflow)
    print(stock.quarterly_cashflow)




In [21]:
get_yfinance("NVDA")

{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building and operating metaverse and 3D internet applications. The Compute & Networking segment co